In [1]:
import re
import random
from typing import Annotated
from dotenv import load_dotenv
from pydantic import BaseModel
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
import gradio as gr


In [2]:
load_dotenv(override=True)

True

In [3]:
def add_value(left, right):
    if right is not None:
        return right
    return left

class State(BaseModel):
    email: Annotated[str, add_value] = ""
    is_valid: Annotated[bool, add_value] = False
    otp: Annotated[str, add_value] = ""
    user_otp: Annotated[str, add_value] = ""
    otp_verified: Annotated[bool, add_value] = False


In [4]:
graph_builder = StateGraph(State)

In [5]:
checkpointer = MemorySaver()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [6]:
# Node that asks GPT-4o to validate the email and return a boolean
# Returns dict to merge into state

def validate_email(state: State):
    prompt = (
        "You are an email validator. Reply with only 'true' or 'false'. "
        "If the email is valid RFC 5322 style and realistic, reply true; otherwise false. "
        f"Email: {state.email}"
    )
    resp = llm.invoke(prompt)
    decision = str(resp.content).strip().lower()
    is_valid = decision.startswith("t")
    return {"email": state.email, "is_valid": is_valid}


# Node that generates a 6-digit OTP once per thread after email is valid

def generate_otp(state: State):
    if not state.is_valid:
        return {"is_valid": False}
    if state.otp:
        return {"otp": state.otp}
    prompt = (
        "Generate a 6 digit numeric one-time password. "
        "Return ONLY the digits, no spaces, no text."
    )
    resp = llm.invoke(prompt)
    text = str(resp.content).strip()
    match = re.search(r"\d{6}", text)
    otp_val = match.group(0) if match else f"{random.randint(0, 999999):06d}"
    return {"otp": otp_val}


# Node that compares user provided OTP with stored OTP

def verify_otp(state: State):
    if not state.otp or not state.user_otp:
        return {"otp_verified": False}
    return {"otp_verified": state.user_otp == state.otp}

In [7]:
graph_builder.add_node("validate_email", validate_email)
graph_builder.add_node("generate_otp", generate_otp)
graph_builder.add_node("verify_otp", verify_otp)

In [8]:
# Wire the graph: START -> validate_email -> generate_otp -> verify_otp -> END
graph_builder.add_edge(START, "validate_email")
graph_builder.add_edge("validate_email", "generate_otp")
graph_builder.add_edge("generate_otp", "verify_otp")
graph_builder.add_edge("verify_otp", END)

graph = graph_builder.compile(checkpointer=checkpointer)

In [9]:
config = {"configurable": {"thread_id": "1"}}

In [10]:
def submit_email(user_email: str, user_otp: str):
    initial_state = State(email=user_email, user_otp=user_otp)
    result_state = graph.invoke(initial_state, config=config)

    if not result_state["is_valid"]:
        return "อีเมลไม่ถูกต้อง กรุณากรอกใหม่"

    if not result_state["otp_verified"]:
        # Show OTP here for demo; in real use, send via email/SMS
        return f"OTP ถูกสร้างแล้ว: {result_state['otp']} กรุณากรอกในช่อง OTP"

    return "OTP ถูกต้อง ยืนยันสำเร็จ"

In [11]:


# Simple UI for collecting the email from the user
gr.Interface(
    fn=submit_email,
    inputs=[
        gr.Textbox(label="กรอกอีเมล"),
        gr.Textbox(label="กรอก OTP (ถ้ามี)")
    ],
    outputs=gr.Textbox(label="ผลลัพธ์"),
    title="LangGraph Email + OTP",
    description="ใส่อีเมลเพื่อสร้าง OTP แล้วกรอก OTP เดิมเพื่อตรวจสอบ",
).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
